In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import icecream as ic

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.utils as vutils
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import wandb
from PIL import Image

e:\Programe\Anaconda\envs\Pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
etichete_proba = torch.LongTensor(np.random.randint(0, 2, 64))
etichete_proba = torch.LongTensor([0, 0, 1, 1])
print(etichete_proba)

tensor([0, 0, 1, 1])
